# General surfaces' math

$S: (u, v) \to (x, y, z)$ 

$S(u, v) = [x(u, v), y(u, v), z(u, v)]$

$S_u = \frac{\partial S}{\partial u}$, $S_v = \frac{\partial S}{\partial v}$

Curve on surface:

$C: (t) \to (u,v)$ 

$C(t) = [u(t), v(t)]$

$T_C = S_u \frac{du}{dt} + S_v \frac{dv}{dt}$

Tangent plane:

$T_S(u,v) = S + \lambda S_u + \mu S_v$

Normal vector:

$N_S(u,v) = \frac{S_u \times S_v}{|S_u \times S_v|}$


# Pipes math
Directrix (spine): $A(t) = [x(t), y(t), z(t)]$

Generatrix (bevel): $C(s, t) = [x(s, t), y(s, t), 0]$, for uniform profile -- $C(s) = [x(s), y(s), 0]$

Generatrix frame: $Z(t) = T_A = A'(t)/|A'(t)|$ -- along spine tangent, $X(t), Y(t)$ -- orthogonal, e.g. $X = N_A, Y = B_A$ or $X = Z \times e_z,  Y = X \times Z$

Surface: $S(u, v) = A(u) + C(v, u)_x X(u) + C(v, u)_y Y(u)$

In [1]:
%matplotlib widget

from itertools import product
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt

from myutils import eval, vec, curv, Plt

In [2]:
t, s, u, v, r, a, b, c, d = sp.symbols("t, s, u, v, r, a, b, c, d")
P, P0, P1, P2, P3 = vec.symbols('P P_0 P_1 P_2 P_3')

# Equations

In [3]:
Bezier1 = lambda t, p0, p1: (1-t) * p0 + t * p1
Bezier2 = lambda t, p0, p1, p2: (1-t)**2 * p0 + 2 * t * (1-t) * p1 + t**2 * p2
Bezier3 = lambda t, p0, p1, p2, p3: (1-t)**3 * p0 + 3 * t * (1-t)**2 * p1 + 3 * t**2 * (1-t) * p2 + t**3 * p3
Ellipse = lambda t, a, b: vec.vector(a * sp.cos(t * 2 * sp.pi), b * sp.sin(t * 2 * sp.pi), 0)
Circle = lambda t, r: vec.vector(r * sp.cos(t * 2 * sp.pi), r * sp.sin(t * 2 * sp.pi), 0)
Cosine = lambda t: vec.vector(t, sp.cos(t * 2 * sp.pi), 0)

In [4]:
Pipe = lambda Qa, Qc, X, Y: Qa + X * Qc[0] + Y * Qc[1]

def pipe_frame(S):
    Sdu = sp.diff(S, u)
    Sdv = sp.diff(S, v)
    Suxv = Sdu.cross(Sdv)
    N = Suxv / vec.length(Suxv)
    return Sdu, Sdv, N

# Geometry

Vetically-aligned spine frame: X axis parallel to world XY, Y axis approximately "up"

In [5]:
def spine_frame(Aa, u):
    Z = sp.simplify(vec.normalize(sp.diff(Aa, u)))         # spine tangent direction
    X = sp.simplify(vec.normalize(Z.cross(vec.vector(0, 0, 1))))
    Y = sp.simplify(X.cross(Z))
    return X, Y, Z


In [6]:
fig = plt.figure(figsize=(12, 6))
plot1 = Plt(fig.add_subplot(1, 2, 1, projection='3d'))
plot2 = Plt(fig.add_subplot(1, 2, 2, projection='3d'))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
plot1.clear()
plot2.clear()

In [8]:
points = (vec.vector(0, 0, 0), vec.vector(0, 1, 1), vec.vector(1, 1, 0))
Qa = Bezier2(u, *points) 
X, Y, Z = spine_frame(Qa, u)
Qc = Circle(v, 0.2)
S = Pipe(Qa, Qc, X, Y)

In [9]:
plot1.curve(Qa, u, color='c')
tt = np.arange(0, 1, 0.125)
pnts = eval(Qa, u=tt)
plot1.vectors(pnts, eval(X, u=tt) * 0.1, color='r')
plot1.vectors(pnts, eval(Y, u=tt) * 0.1, color='g')
plot1.vectors(pnts, eval(Z, u=tt) * 0.1, color='b')

In [11]:
plot2.surface(S, u, v, color='c', alpha=0.5)

Su, Sv, N = pipe_frame(S)
uu = np.linspace(0, 1, 4)
vv = np.linspace(0, 1, 4)
pnts = eval(S, u=uu, v=vv)
plot2.vectors(pnts, eval(Su, u=uu, v=vv) * 0.1, color='r')
plot2.vectors(pnts, eval(Sv, u=uu, v=vv) * 0.1, color='g')
plot2.vectors(pnts, eval(N, u=uu, v=vv) * 0.1, color='b')

# Reverse projection

Projecting points from fabric surface to pipe.

$P_{flat} \to S \to u_{proj}, v_{proj}, V_{displacement}$ 

Flat spine curve:

$Q_z = 0$, $Y_{spine} = e_z$, $X_{spine} = distance(P_{flat}, Q_{flat})$

Solution for flat curve:
1. find closest point on spine $Qa \to u_{proj}, x_{distance}$
2. solve $Qc(v)_x = x_{distance} \to v_{proj}$
3. $P_s = S(u_{proj}, v_{proj}) \to V_{displ}$ 

## Projecting to Qa

Distance vector: $V_{dist}(u) = Q_{flat}(u) - P_{flat}$

Squared distance: $V_{dist} \cdot V_{dist}(u)$

Derivative: $\frac{dV}{du}(u)$

Solve: $\frac{dV}{du}(u) = 0 \to u_{proj}$

In [58]:
def find_closest(Qa, u, p):
    Vdst = Qa - p
    Vdst_sq = Vdst.dot(Vdst)
    Vdu = sp.diff(Vdst_sq, u)
    solution = sp.solve(Vdu, u)
    return solution[0] 

### Manageable

$Q(u) = A u^2 + B u + C$

$V = Q(u) - P = A u^2 + B u + C$

* $A = (P_0 - P_1) + (P_2 - P_1)$
* $B = -2 (P_0 - P_1)$
* $C = P_0$
* $D = P_0 - P$

$V \cdot V = (A \cdot A) u^4  + (2 A \cdot B) u^3 + (2 A \cdot D + B \cdot B) u^2  + (2 B \cdot D) u + D \cdot D$

$(V \cdot V)/du =  4 (A \cdot A) u^3 + 6 (A \cdot B) u^2 + 2 (2 A \cdot D + B \cdot B) u + 2 B \cdot D$

Equation

$2 (A \cdot A) u^3 + 3 (A \cdot B) u^2 + (2 A \cdot D + B \cdot B) u + B \cdot D = 0$

$a u^3 + b u^2 + c u + d = 0$

* $a = 2 (A \cdot A)$
* $b = 3 (A \cdot B)$
* $c = (2 A \cdot D + B \cdot B)$
* $d = B \cdot D$

$u'^3 + p u' + q = 0$

* $u' = u + b/3a$
* $p = \frac{3 ac - b^2}{3 a^2}$
* $q = \frac{2 b^3 - 9 abc + 27 a^2d}{27a^3}$

Cardano formula, if $4 p^3 + 27q^2 > 0$

$u' = 
\sqrt[3]{  
    -\frac{q}{2}
    +\sqrt{
         (\frac{q}{2})^2 + (\frac{p}{3})^3   
     }
}
+
\sqrt[3]{  
    -\frac{q}{2}
    -\sqrt{
         (\frac{q}{2})^2 + (\frac{p}{3})^3   
     }
}
$

$u_{proj} = u' - b/3a$

## Projecting to Qc

Circle generatrix

$C_x(v) = r cos(v 2 \pi)$

$r cos(v 2 \pi) = dist$

If $-1 < d/r < 1$

$v_{proj} = 1 - \frac{acos(d/r)}{2\pi}$

